In [16]:
import pandas as pd
import re


In [63]:
scraped= pd.read_csv('Hotel Location Table Mapping.csv',encoding='latin1')
hotels=pd.read_csv('Hotel Location.csv',encoding='latin1')
old=pd.read_csv('old.csv',encoding='latin1')
pivot=pd.read_csv('pivot.csv',encoding='latin1')

In [64]:
def clean_text(text):
    """
    This function uses regular expressions to 
        - remove links characters
        - remove html characters,
        - remove escape sequences,
        - remove capitalization,
        - any extra white space from each text and then converts them to lowercase.

    Input:
    text: original text
          datatype: string

    Output:
    texts: modified text
           datatype: string
    """
    # replace links with url-web
    pattern_url = 'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
    subs_url = 'url-web'
    text = re.sub(pattern_url, subs_url, text)
    # replace the html characters with " "
    text = re.sub('<.*?>#()&', ' ', text)
    text = re.sub('&', ' ', text)
#     text = re.sub('(', ' ', text)
#     text = re.sub(')', ' ', text)
    
    text=re.sub('Nan', ' ', text)
    # replace escape sequence with space
    escape_seq = ["\r", "\n",'\a', '\b', '\f', "\`", '\”', '\t']
    for i in escape_seq:
        text = re.sub(i, " ", text)
    # Remove Capitalization
    text = text.lower()
    # plit and join the words
    text=' '.join(text.split())
    
    return text

In [65]:
scraped['description'] = scraped['description'].astype(str).apply(clean_text)
scraped['inclusions'] = scraped['inclusions'].astype(str).apply(clean_text)
scraped['hotel_name'] = scraped['hotel_name'].astype(str).apply(clean_text)
scraped['title'] = scraped['title'].astype(str).apply(clean_text)

In [66]:
scraped['corpus'] = ''
corpus = []
columns = ['title','hotel_name','description', 'inclusions']
for i in range(0, len(scraped['scraped_url'])):
    words = ''
    for col in columns:
        words = words + scraped.iloc[i][col] + " "        
    corpus.append(words)
scraped['corpus'] = corpus
#scrapy.set_index('scraped_url', inplace=True)
scraped.drop(columns=['title', 'description', 'inclusions', 'hotel_name','agency_slug','country'], inplace=True)
scraped.head()




,scraped_url,hotel_mapped,corpus
0,https://www.flightcentre.co.za/product/14197346,NaN,"adaaran select meedhupparu nan your maldives holiday package includes: return flights from johannesburg approximate taxes return domestic flight speedboat transfers 7 nights' accommodation at the 4-star adaaran select meedhupparu in a beachfront villa all meals selected alcoholic and non-alcoholic drinks daily exclusions items of a personal nature, items not mentioned, gratuities and porterage, meals drinks not mentioned, tours/sightseeing not mentioned, car hire with the promise of an incomparable holiday in the tropics, adaaran select meedhupparu one of the best maldives luxury resorts offers blissful accommodation for the discerning traveller to the maldives. bask under the healing sunshine and soak up the alluring sights and sounds of a truly enchanting island in the tropics. adaaran select meedhupparu offers tranquil settings brimming with luxury. settle into a holiday full of breathtaking experiences that will leave you with memories to last a lifetime. your maldives holiday package includes: return flights from johannesburg approximate taxes return domestic flight speedboat transfers 7 nights' accommodation at the 4-star adaaran select meedhupparu in a beachfront villa all meals selected alcoholic and non-alcoholic drinks daily"
1,https://www.flightcentre.co.za/product/5415249,NaN,"family fun on the wild coast nan your eastern cape holiday package includes: return flights from johannesburg to durban approximate taxes 3 days car hire (kia picanto or similar) 3 nights' midweek accommodation on the wild coast at the 4-star wild coast sun in a garden facing luxury room breakfast daily bonus: complimentary access to wild waves water park please note: 2 children eat breakfast and stay free when sharing with adults weekend upgrade: from r660 per adult exclusions transfers, items of a personal nature, items not mentioned, gratuities and porterage, meals and drinks not mentioned, tours/sightseeing not mentioned less than 2 hours from durban. you can't beat the location of the wild coast sun hotel, situated in the coastal hub of port edward, less than a two-hour scenic drive along the south coast from durban's king shaka international airport. this family-friendly accommodation is just the place if you're looking for that relaxing room with a view, with all the modern facilities one could ever need. moms and dads can soak up the sun watching the kids play at the wild waves water park, while couples can slip away to the tranquil oasis of the mangwanani african day spa. your eastern cape holiday package includes: return flights from johannesburg to durban approximate taxes 3 days car hire (kia picanto or similar) 3 nights' midweek accommodation on the wild coast at the 4-star wild coast sun in a garden facing luxury room breakfast daily bonus: complimentary access to wild waves water park please note: 2 children eat breakfast and stay free when sharing with adults weekend upgrade: from r660 per adult"
2,https://www.flightcentre.co.za/product/16385011,NaN,"azuri residences by life in blue nan your mauritius holiday package includes: return direct flights from durban approximate taxes return transfers 5 nights' accommodation at the 4-star azuri residences by life in blue in a deluxe apartment on a room only basis please note: price is based on 4 people travelling together exclusions items of a personal nature, items not mentioned, gratuities and porterage, meals and drinks not mentioned, tours/sightseeing not mentioned, car hire this coastal village on the north-eastern coast of the island in roches noires, lined by an infinite turquoise sea and the blue sky, will tinge your holidays with picturesque shades of blue. opt for our high-end seafront accommodation and enjoy your intimacy within an elegant setting. kindle your senses and experience la dolce vita through a memorable escape to mauritius where you will benefit from services available at yo

In [68]:
# Verify Actions
pd.set_option('display.max_colwidth', -1)      # set to display full text in df
scraped['corpus'].head(15)

C:\Users\user\AppData\Local\Temp\ipykernel_8628\1971666159.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)      # set to display full text in df


0     adaaran select meedhupparu nan your maldives holiday package includes return flights from johannesburg approximate taxes return domestic flight speedboat transfers 7 nights accommodation at the 4star adaaran select meedhupparu in a beachfront villa all meals selected alcoholic and nonalcoholic drinks daily exclusions items of a personal nature items not mentioned gratuities and porterage meals drinks not mentioned tourssightseeing not mentioned car hire with the promise of an incomparable holiday in the tropics adaaran select meedhupparu one of the best maldives luxury resorts offers blissful accommodation for the discerning traveller to the maldives bask under the healing sunshine and soak up the alluring sights and sounds of a truly enchanting island in the tropics adaaran select meedhupparu offers tranquil settings brimming with luxury settle into a holiday full of breathtaking experiences that will leave you with memories to last a lifetime your maldives holiday package inclu

In [36]:
hotels['Name'] = hotels['Name'].astype(str).apply(clean_text)
#scraped['title'] = scraped['title'].astype(str).apply(clean_text)

In [45]:
def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("#",' ',str(text))
    text = re.sub("@",'',str(text))
    
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]#", "", str(text))
    
    return text

In [48]:
hotels['Name'] = hotels['Name'].astype(str).apply(clean)

In [49]:
import string

In [51]:
def remove_punctuation(post):
    return ''.join([l for l in post if l not in string.punctuation])

In [52]:
hotels['Name'] = hotels['Name'].astype(str).apply(remove_punctuation)

In [54]:
hotels['Name'].head(30)

0     41                                
1      2 andrews  port douglas         
2      2 hastings street                
3     com                               
4     the dunstane                      
5      beach bed bar                    
6      michigan inn lodge               
7      samui haus                       
8      the hub west                     
9      white patong                     
10    hom hotel kudus by horison group  
11    home boutique hotel               
12    home budget hotel                 
13    hua lamphong                      
14    inn                               
15    m2 villa                          
16    thepad hotel resort               
17    06 central hostel buenos aires    
18    08028 apartments                  
19    1 baron motel                     
20    1 bedroom china town condo        
21    1 borneo tower b service apartment
22    1 golf place                      
23    1 hotel central park              
24    1 hotel so

In [117]:
list_hotels=hotels['Name'].astype(str).tolist

In [118]:
list_hotels

<bound method IndexOpsMixin.tolist of 0         41                       
1         #2 Andrews  Port Douglas
2         #2 Hastings Street       
3         .Com                     
4         ?The Dunstane            
              ...                  
145626    Zwahili Game Lodge & Spa 
145627    Zyanrooms                
145628    Zz House Chiang Mai      
145629    Zzziesta Chiangmai       
145630    Zzzip Guest House        
Name: Name, Length: 145631, dtype: object>

In [116]:
list_hotels.astype(str)


AttributeError: 'function' object has no attribute 'astype'

In [59]:
hotels.head()

,Name,Address1,Address2,City,StateProvince,PostalCode,Country,Latitude,Longitude,AirportCode,PropertyCurrency,StarRating,Location
0,41,41 Buckingham Palace Road,NaN,London,NaN,SW1W 0PS,GB,51.49830,-0.14372,LHR,GBP,5.0,Near Buckingham Palace
1,2 andrews  port douglas,#2 Andrews Close,NaN,Port Douglas,QL,4877,AU,-16.51689,145.46770,CNS,AUD,5.0,Near Four Mile Beach Park
2,2 hastings street,2 Hastings Street,NaN,Noosa Heads,QL,4567,AU,-26.38616,153.08722,MCY,AUD,4.0,In Noosa Heads (Noosa)
3,com,Albena,NaN,Albena,NaN,9620,BG,43.37024,28.07389,VAR,EUR,3.0,In Albena
4,the dunstane,4 West Coates & 5 Hampton Terrace,Haymarket,Edinburgh,NaN,EH12 5JQ,GB,55.94571,-3.22880,EDI,GBP,4.0,Near Murrayfield Stadium


In [67]:
scraped['corpus'] = scraped['corpus'].astype(str).apply(remove_punctuation)

In [70]:
import nltk
from nltk.tokenize import word_tokenize, TreebankWordTokenizer

In [73]:
tokenizer = TreebankWordTokenizer()
scraped['tokens'] = scraped['corpus'].apply(tokenizer.tokenize)

In [74]:
scraped.head()

,scraped_url,hotel_mapped,corpus,tokens
0,https://www.flightcentre.co.za/product/14197346,NaN,adaaran select meedhupparu nan your maldives holiday package includes return flights from johannesburg approximate taxes return domestic flight speedboat transfers 7 nights accommodation at the 4star adaaran select meedhupparu in a beachfront villa all meals selected alcoholic and nonalcoholic drinks daily exclusions items of a personal nature items not mentioned gratuities and porterage meals drinks not mentioned tourssightseeing not mentioned car hire with the promise of an incomparable holiday in the tropics adaaran select meedhupparu one of the best maldives luxury resorts offers blissful accommodation for the discerning traveller to the maldives bask under the healing sunshine and soak up the alluring sights and sounds of a truly enchanting island in the tropics adaaran select meedhupparu offers tranquil settings brimming with luxury settle into a holiday full of breathtaking experiences that will leave you with memories to last a lifetime your maldives holiday package includes return flights from johannesburg approximate taxes return domestic flight speedboat transfers 7 nights accommodation at the 4star adaaran select meedhupparu in a beachfront villa all meals selected alcoholic and nonalcoholic drinks daily,"[adaaran, select, meedhupparu, nan, your, maldives, holiday, package, includes, return, flights, from, johannesburg, approximate, taxes, return, domestic, flight, speedboat, transfers, 7, nights, accommodation, at, the, 4star, adaaran, select, meedhupparu, in, a, beachfront, villa, all, meals, selected, alcoholic, and, nonalcoholic, drinks, daily, exclusions, items, of, a, personal, nature, items, not, mentioned, gratuities, and, porterage, meals, drinks, not, mentioned, tourssightseeing, not, mentioned, car, hire, with, the, promise, of, an, incomparable, holiday, in, the, tropics, adaaran, select, meedhupparu, one, of, the, best, maldives, luxury, resorts, offers, blissful, accommodation, for, the, discerning, traveller, to, the, maldives, bask, under, the, healing, sunshine, and, soak, up, ...]"
1,https://www.flightcentre.co.za/product/5415249,NaN,family fun on the wild coast nan your eastern cape holiday package includes return flights from johannesburg to durban approximate taxes 3 days car hire kia picanto or similar 3 nights midweek accommodation on the wild coast at the 4star wild coast sun in a garden facing luxury room breakfast daily bonus complimentary access to wild waves water park please note 2 children eat breakfast and stay free when sharing with adults weekend upgrade from r660 per adult exclusions transfers items of a personal nature items not mentioned gratuities and porterage meals and drinks not mentioned tourssightseeing not mentioned less than 2 hours from durban you cant beat the location of the wild coast sun hotel situated in the coastal hub of port edward less than a twohour scenic drive along the south coast from durbans king shaka international airport this familyfriendly accommodation is just the place if youre looking for that relaxing room with a view with all the modern facilities one could ever need moms and dads can soak up the sun watching the kids play at the wild waves water park while couples can slip away to the tranquil oasis of the mangwanani african day spa your eastern cape holiday package includes return flights from johannesburg to durban approximate taxes 3 days car hire kia picanto or similar 3 nights midweek accommodation on the wild coast at the 4star wild coast sun in a garden facing luxury room breakfast daily bonus complimentary access to wild waves water park please note 2 children eat breakfast and stay free when sharing with adults weekend upgrade from r660 per adult,"[family, fun, on, the, wild, coast, nan, your, eastern, cape, holiday, package, includes, return, flights, from, johannesburg, to, durban, approximate, taxes, 3, days, car, hire, kia, picanto, or, simi

In [78]:
scraped['tokens'][:1].astype(str)

0    ['adaaran', 'select', 'meedhupparu', 'nan', 'your', 'maldives', 'holiday', 'package', 'includes', 'return', 'flights', 'from', 'johannesburg', 'approximate', 'taxes', 'return', 'domestic', 'flight', 'speedboat', 'transfers', '7', 'nights', 'accommodation', 'at', 'the', '4star', 'adaaran', 'select', 'meedhupparu', 'in', 'a', 'beachfront', 'villa', 'all', 'meals', 'selected', 'alcoholic', 'and', 'nonalcoholic', 'drinks', 'daily', 'exclusions', 'items', 'of', 'a', 'personal', 'nature', 'items', 'not', 'mentioned', 'gratuities', 'and', 'porterage', 'meals', 'drinks', 'not', 'mentioned', 'tourssightseeing', 'not', 'mentioned', 'car', 'hire', 'with', 'the', 'promise', 'of', 'an', 'incomparable', 'holiday', 'in', 'the', 'tropics', 'adaaran', 'select', 'meedhupparu', 'one', 'of', 'the', 'best', 'maldives', 'luxury', 'resorts', 'offers', 'blissful', 'accommodation', 'for', 'the', 'discerning', 'traveller', 'to', 'the', 'maldives', 'bask', 'under', 'the', 'healing', 'sunshine', 'and', 'soak

In [139]:
scraped.iloc[:, [2]]

corpus
1     family fun on the wild coast nan your eastern cape holiday package includes return flights from johannesburg to durban approximate taxes 3 days car hire kia picanto or similar 3 nights midweek accommodation on the wild coast at the 4star wild coast sun in a garden facing luxury room breakfast daily bonus complimentary access to wild waves water park please note 2 children eat breakfast and stay free when sharing with adults weekend upgrade from r660 per adult exclusions transfers items of a personal nature items not mentioned gratuities and porterage meals and drinks not mentioned tourssightseeing not mentioned less than 2 hours from durban you cant beat the location of the wild coast sun hotel situated in the coastal hub of port edward less than a twohour scenic drive along the south coast from durbans king shaka international airport this familyfriendly accommodation is just the place if youre looking for that relaxing room with a view with all the modern facilities one could ever need moms and dads can soak up the sun watching the kids play at the wild waves water park while couples can slip away to the tranquil oasis of the mangwanani african day spa your eastern cape holiday package includes return flights from johannesburg to durban approximate taxes 3 days car hire kia picanto or similar 3 nights midweek accommodation on the wild coast at the 4star wild coast sun in a garden facing luxury room breakfast daily bonus complimentary access to wild waves water park please note 2 children eat breakfast and stay free when sharing with adults weekend upgrade from r660 per adult                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

scraped_url  \
0     https://www.flightcentre.co.za/product/14197346                                                                            
1     https://www.flightcentre.co.za/product/5415249                                                                             
2     https://www.flightcentre.co.za/product/16385011                                                                            
3     https://www.flightcentre.co.za/product/16323810                                                                            
4     https://www.flightcentre.co.za/product/13250621                                                                            
...                                               ...                                                                            
8376  https://www.thompsons.co.za/holiday-packages/west-coast-flowers--cederberg-selfdrive-package-8-days7-nights/PP-TH3118366   
8377  https://packages.travelstart.co.za/Zilwa-Attitude-Hotel-13501                                                              
8378  https://packages.travelstart.co.za/Zilwa-Attitude-Hotel-13501                                                              
8379  NaN                                                                                                                        
8380  NaN                                                                                                                        

      hotel_mapped  \
0    NaN             
1    NaN             
2    NaN             
3    NaN             
4    NaN             
...   ..             
8376 NaN             
8377 NaN             
8378 NaN             
8379 NaN             
8380 NaN             

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [119]:
test_list1=list_hotels
test_list2=['adaaran', 'select', 'meedhupparu', 'nan', 'your', 'maldives', 'holiday', 'package', 'includes', 'return', 'flights', 'from', 'johannesburg', 'approximate', 'taxes', 'return', 'domestic', 'flight', 'speedboat', 'transfers', '7', 'nights', 'accommodation', 'at', 'the', '4star', 'adaaran', 'select', 'meedhupparu', 'in', 'a', 'beachfront', 'villa', 'all', 'meals', 'selected', 'alcoholic', 'and', 'nonalcoholic', 'drinks', 'daily', 'exclusions', 'items', 'of', 'a', 'personal', 'nature', 'items', 'not', 'mentioned', 'gratuities', 'and', 'porterage', 'meals', 'drinks', 'not', 'mentioned', 'tourssightseeing', 'not', 'mentioned', 'car', 'hire', 'with', 'the', 'promise', 'of', 'an', 'incomparable', 'holiday', 'in', 'the', 'tropics', 'adaaran', 'select', 'meedhupparu', 'one', 'of', 'the', 'best', 'maldives', 'luxury', 'resorts', 'offers', 'blissful', 'accommodation', 'for', 'the', 'discerning', 'traveller', 'to', 'the', 'maldives', 'bask', 'under', 'the', 'healing', 'sunshine', 'and', 'soak', 'up', 'the', 'alluring', 'sights', 'and', 'sounds', 'of', 'a', 'truly', 'enchanting', 'island', 'in', 'the', 'tropics', 'adaaran', 'select', 'meedhupparu', 'offers', 'tranquil', 'settings', 'brimming', 'with', 'luxury', 'settle', 'into', 'a', 'holiday', 'full', 'of', 'breathtaking', 'experiences', 'that', 'will', 'leave', 'you', 'with', 'memories', 'to', 'last', 'a', 'lifetime', 'your', 'maldives', 'holiday', 'package', 'includes', 'return', 'flights', 'from', 'johannesburg', 'approximate', 'taxes', 'return', 'domestic', 'flight', 'speedboat', 'transfers', '7', 'nights', 'accommodation', 'at', 'the', '4star', 'adaaran', 'select', 'meedhupparu', 'in', 'a', 'beachfront', 'villa', 'all', 'meals', 'selected', 'alcoholic', 'and', 'nonalcoholic', 'drinks', 'daily']
#res = list(set([ele1 for sub1 in lis for ele1 in lis2 if ele1 in sub1]))
res = [ele2 for ele2 in test_list2 if any(ele2 in ele1 for ele1 in test_list1)]
print(str(res))

TypeError: 'method' object is not iterable

In [84]:
def check(strings, substrings):
    for substring in substrings:
        if any(substring in s for s in strings):
            return print(s)
        

In [124]:
# initializing lists

test_list1=['adaaran', 'select', 'meedhupparubig', 'nan', 'your', 'maldives', 'holiday', 'package', 'includes', 'return', 'flights', 'from', 'johannesburg', 'approximate', 'taxes', 'return', 'domestic', 'flight', 'speedboat', 'transfers', '7', 'nights', 'accommodation', 'at', 'the', '4star', 'adaaran', 'select', 'meedhupparu', 'in', 'a', 'beachfront', 'villa', 'all', 'meals', 'selected', 'alcoholic', 'and', 'nonalcoholic', 'drinks', 'daily', 'exclusions', 'items', 'of', 'a', 'personal', 'nature', 'items', 'not', 'mentioned', 'gratuities', 'and', 'porterage', 'meals', 'drinks', 'not', 'mentioned', 'tourssightseeing', 'not', 'mentioned', 'car', 'hire', 'with', 'the', 'promise', 'of', 'an', 'incomparable', 'holiday', 'in', 'the', 'tropics', 'adaaran', 'select', 'meedhupparu', 'one', 'of', 'the', 'best', 'maldives', 'luxury', 'resorts', 'offers', 'blissful', 'accommodation', 'for', 'the', 'discerning', 'traveller', 'to', 'the', 'maldives', 'bask', 'under', 'the', 'healing', 'sunshine', 'and', 'soak', 'up', 'the', 'alluring', 'sights', 'and', 'sounds', 'of', 'a', 'truly', 'enchanting', 'island', 'in', 'the', 'tropics', 'adaaran', 'select', 'meedhupparu', 'offers', 'tranquil', 'settings', 'brimming', 'with', 'luxury', 'settle', 'into', 'a', 'holiday', 'full', 'of', 'breathtaking', 'experiences', 'that', 'will', 'leave', 'you', 'with', 'memories', 'to', 'last', 'a', 'lifetime', 'your', 'maldives', 'holiday', 'package', 'includes', 'return', 'flights', 'from', 'johannesburg', 'approximate', 'taxes', 'return', 'domestic', 'flight', 'speedboat', 'transfers', '7', 'nights', 'accommodation', 'at', 'the', '4star', 'adaaran', 'select', 'meedhupparu', 'in', 'a', 'beachfront', 'villa', 'all', 'meals', 'selected', 'alcoholic', 'and', 'nonalcoholic', 'drinks', 'daily']
test_list2=['crime','sister','meedhupparu','cousin','aunt']
 # as Substring
#res = [ele2 for ele2 in test_list2 if any(ele2 in ele1 for ele1 in test_list1)]
res = [ele1 for ele1 in test_list1 if any(ele2 in ele1 for ele2 in test_list2)]

 
# printing result
print(str(res))

['meedhupparubig', 'meedhupparu', 'meedhupparu', 'meedhupparu', 'meedhupparu']


In [120]:
test_list2

['adaaran',
 'select',
 'meedhupparu',
 'nan',
 'your',
 'maldives',
 'holiday',
 'package',
 'includes',
 'return',
 'flights',
 'from',
 'johannesburg',
 'approximate',
 'taxes',
 'return',
 'domestic',
 'flight',
 'speedboat',
 'transfers',
 '7',
 'nights',
 'accommodation',
 'at',
 'the',
 '4star',
 'adaaran',
 'select',
 'meedhupparu',
 'in',
 'a',
 'beachfront',
 'villa',
 'all',
 'meals',
 'selected',
 'alcoholic',
 'and',
 'nonalcoholic',
 'drinks',
 'daily',
 'exclusions',
 'items',
 'of',
 'a',
 'personal',
 'nature',
 'items',
 'not',
 'mentioned',
 'gratuities',
 'and',
 'porterage',
 'meals',
 'drinks',
 'not',
 'mentioned',
 'tourssightseeing',
 'not',
 'mentioned',
 'car',
 'hire',
 'with',
 'the',
 'promise',
 'of',
 'an',
 'incomparable',
 'holiday',
 'in',
 'the',
 'tropics',
 'adaaran',
 'select',
 'meedhupparu',
 'one',
 'of',
 'the',
 'best',
 'maldives',
 'luxury',
 'resorts',
 'offers',
 'blissful',
 'accommodation',
 'for',
 'the',
 'discerning',
 'traveller',
 